In [31]:
%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones.ipynb

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [33]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [34]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [35]:
tasa, size, ratio, error = actividad_productiva(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = busqueda_trabajo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
desempleo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_derechos_laborales(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_descuento = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = participacion_trabajo_remunerado(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
horas_remuneradas = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = inactividad(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
ratio_inactividad = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = actividad(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
ratio_actividad = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_aguinaldo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_aguinaldo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_obra_social(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_obra_social = pd.concat([tasa, size, ratio, error], axis=1)

In [36]:
insercion_laboral = (ratio_actividad['Actividad'] + \
                    empleo_asalariado_con_descuento['Acceso a derechos laborales'] + \
                    horas_remuneradas['Participacion trabajo remunerado']) / 3

In [37]:
tabla_pob_ocupada_asalariada = calcular_tabla_poblacion_ocupada_asalariada(df_people_pool, tipo='Urbano', base='Individual')

### Analisis dfs

In [38]:
# cod_provincia
dict_cod_provincia = {2: "CABA",
 6: "Buenos Aires",
 10: "Catamarca",
 14: "Córdoba",
 18: "Corrientes",
 22: "Chaco",
 26: "Chubut",
 30: "Entre Ríos",
 34: "Formosa",
 38: "Jujuy",
 42: "La Pampa",
 46: "La Rioja",
 50: "Mendoza",
 54: "Misiones",
 58: "Neuquén",
 62: "Río Negro",
 66: "Salta",
 70: "San Juan",
 74: "San Luis",
 78: "Santa Cruz",
 82: "Santa Fe",
 86: "Santiago del Estero",
 90: "Tucumán",
 94: "TdF"}

In [39]:
variables = [empleo, desempleo, empleo_asalariado_con_descuento, horas_remuneradas, 
             ratio_inactividad, ratio_actividad, empleo_asalariado_con_aguinaldo, empleo_asalariado_con_obra_social, insercion_laboral] 

for variable in variables:
    variable.index = variable.index.map(dict_cod_provincia)


### Cargo cosas para plots

In [40]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

In [41]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

### Plot insercion laboral

In [42]:
def create_bar_chart(df_list, error_cols, scatter_cols, colors, file_path, filename):
    bar_traces = []
    scatter_traces = []

    for i, df in enumerate(df_list):
        error_y = dict(
            type='data',
            symmetric=False,
            array=df['LS'] - df[error_cols[i]],
            arrayminus=df[error_cols[i]] - df['LI'],
            visible=True,
            color='black',
            thickness=0.7
        )
        
        bar_trace = go.Bar(
            x=df.index,
            y=df[error_cols[i]],
            name='T' + str(i + 1) + '. ' + error_cols[i],
            marker_color=colors[i],
            error_y=error_y
        )
        bar_traces.append(bar_trace)

        scatter_trace = go.Scatter(
            x=df.index,
            y=df[scatter_cols[i]],
            name='T' + str(i + 1) + '. ER',
            mode='markers',
            marker=dict(
                color='black',
                size=8
            ),
            yaxis='y2'
        )
        scatter_traces.append(scatter_trace)

    layout = go.Layout(
        title='Insercion laboral',
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(y=1.1, orientation='h'),
        yaxis=dict(title='Actividad', range=[0, 170]),
        scattermode="group",
        yaxis2=dict(title='ER', overlaying='y', side='right', range=[0, 70])
    )

    fig = go.Figure(data=bar_traces + scatter_traces, layout=layout)

    fig.add_shape(
        type='line',
        x0=df_list[0].index[0],
        y0=100,
        x1=df_list[0].index[-1],
        y1=100,
        line=dict(
            color='black',
            width=1,
            dash='dash'
        ),
    )

    plot(fig, filename=file_path + filename, auto_open=False)



In [43]:
def create_componente_insercion_laboral(sorted_data, file_path, filename):
    # Crear la barra de trazado
    bar_trace1 = go.Bar(
        x=sorted_data.index,
        y=sorted_data,
        name=' ',
        marker_color=fundar_colores_primarios[0]
    )

    # Crear el layout
    layout = go.Layout(
        title='Indicador componente insercion laboral',
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)'
    )

    # Crear la figura
    fig = go.Figure(data=[bar_trace1], layout=layout)

    # Agregar la línea de referencia
    fig.add_shape(
        type='line',
        x0=sorted_data.index[0],
        y0=100,
        x1=sorted_data.index[-1],
        y1=100,
        line=dict(
            color='black',
            width=1,
            dash='dash'
        ),
    )

    fig.update_yaxes(range=[0, 90])

    # Guardar el gráfico como un archivo HTML
    plot(fig, filename=file_path + filename, auto_open=False)


In [44]:
def create_stacked_bar_chart(df_list, y_columns, colors, title, file_path, filename):
    bar_traces = []

    for i, df in enumerate(df_list):
        x = df.index
        for j, y_col in enumerate(y_columns):
            y_values = df[y_col]
            
            bar_trace = go.Bar(
                x=x,
                y=y_values,
                name=f'{y_col}',
                marker_color=colors[i]
            )
            bar_traces.append(bar_trace)

    layout = go.Layout(
        title=title,
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(y=1.1, orientation='h'),
        yaxis=dict(title='Cantidad'),
        xaxis=dict(title='Provincia'),
    )

    fig = go.Figure(data=bar_traces, layout=layout)

    plot(fig, filename=file_path + filename, auto_open=False)


In [45]:
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'

sorted_data = insercion_laboral.sort_values(ascending=False)

filename = 'componente_insercion_laboral_16-a-65.html'
create_componente_insercion_laboral(sorted_data, file_path, filename)

df_list = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas]

y_columns = ['Varon', 'Mujer']
colors = [fundar_colores_primarios[0], 
          fundar_colores_primarios[1],
          fundar_colores_primarios[2],
          fundar_colores_primarios[3],
          fundar_colores_primarios[4]
          ]

title = 'Comparación de Varones y Mujeres por Provincia' 
filename = 'tasas_indicadores_insercion_laboral_16-a-65.html'

create_stacked_bar_chart(df_list, y_columns, colors, title, file_path, filename)

df_list = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas]
error_cols = ['Actividad', 'Acceso a derechos laborales', 'Participacion trabajo remunerado']
scatter_cols = ['ER'] * len(df_list)
colors = fundar_colores_primarios[:len(df_list)]
filename = 'insercion_laboral_16-a-65.html'

create_bar_chart(df_list, error_cols, scatter_cols, colors, file_path, filename)